<a href="https://colab.research.google.com/github/tiagochavo87/LCSH_analysis/blob/main/analysis_of_cma_lcshsV2(IN_DESENV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
import pandas as pd

# Read the data from the CSV file
data = pd.read_csv("LOH.csv", sep=";")

# Apply a filter to exclude chromosomes X and Y
data = data.loc[(data['Chrom'] != 'X') & (data['Chrom'] != 'Y')]

# Replace dots and convert Tamanho column to integer type
data['Tamanho'] = data['Tamanho'].str.replace('.', '').astype(int)

# Filter data for autosomes with a size greater than 10,000,000
data_autosomes = data.loc[data['Chrom'].str.contains('^([1-9]|[1-9][0-9])$')]
data_autosomes = data_autosomes[data_autosomes['Tamanho'] > 10000000]

# Write the autosomes data to a new CSV file
data_autosomes.to_csv('LCSHs_>_10.csv', sep=';', index=False)

# Filter data for regions with size greater than or equal to 3,000,000
data = data[data['Tamanho'] >= 3000000]

# Filter data for regions with sizes between 3,000,000 and 5,000,000
data_3_5 = data[(data['Tamanho'] > 3000000) & (data['Tamanho'] <= 5000000)]

# Write the 3-5 MB regions data to a new CSV file
data_3_5.to_csv('LCSHs_>_3_<_5.csv', sep=';', index=False)

# Filter data for 3-5 MB regions with a total size of at least 10,000,000 and only one region per file
data_3_5 = data_3_5.groupby('File').filter(lambda x: len(x) == 1 and x['Tamanho'].sum() >= 10000000)

# Filter data for regions with size greater than 5,000,000
data_5 = data[data['Tamanho'] > 5000000]

# Filter data for regions with a total size of at least 10,000,000 and only one chromosome per file
data_5 = data_5.groupby('File').filter(lambda x: len(x.groupby('Chrom')) == 1 and x['Tamanho'].sum() >= 10000000)

# Write the 5 MB regions data to a new CSV file
data_5.to_csv('possibles_UPDs.csv', sep=';', index=False)

# Filter data for cases with at least one LCSH greater than 5 MB
data_5_mb = data[data['Tamanho'] > 5000000]
data_5_mb = data_5_mb.groupby('File').filter(lambda x: x[x['Tamanho'] >= 5000000].shape[0] > 0)

# Print all cases with LCSHs greater than 5 MB to a CSV file
data_5_mb.to_csv('LCSHs_>_5.csv', sep=';', index=False)

# Sum LCSHs greater than 3 MB for each case
sum_lcshs = data[data['Tamanho'] >= 3000000].groupby('File')['Tamanho'].sum()

# Create a new DataFrame with case names and total LCSHs greater than 3 MB
output = pd.DataFrame({'File': sum_lcshs.index, 'Total LCSHs > 3MB': sum_lcshs.values})

# Write the output DataFrame to a CSV file
output.to_csv('total_lcshs.csv', sep=';', index=False)

# Calculate the total size of human autosomal DNA
total_autosomal_dna = 2881000000

# Calculate the proportion of LCSHs greater than 3 MB for each case
proportions = []
for file, size in sum_lcshs.items():
    proportion = size / total_autosomal_dna
    proportions.append({'File': file, 'Total LCSHs > 3MB': size, 'Proportion of LCSHs > 3MB to Autosomal DNA': proportion})

# Create a new DataFrame with the proportion for each case
proportions_df = pd.DataFrame(proportions)

# Write the output DataFrame to a CSV file
proportions_df.to_csv('lcsh_proportions.csv', sep=';', index=False)


<ipython-input-123-6cb181c1a486>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Tamanho'] = data['Tamanho'].str.replace('.', '').astype(int)
<ipython-input-123-6cb181c1a486>:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  data_autosomes = data.loc[data['Chrom'].str.contains('^([1-9]|[1-9][0-9])$')]


This code imports the NumPy and Pandas libraries and uses them to calculate inbreeding coefficients based on certain proportions of data.

The first step is to define an array of values for F (inbreeding coefficient) that the code will use. These values are defined as percentages and then divided by 100.

Next, the code reads in a CSV file containing proportions of data and stores them in a Pandas DataFrame.

Then, the code iterates through each row of the DataFrame and calculates the F value that is closest to the proportion of LCSHs (Library of Congress Subject Headings) greater than 3MB to Autosomal DNA. This is done using NumPy's argmin() function to find the index of the closest F value in the array.

After calculating the closest F value for each row, the code creates a new Pandas DataFrame with the data transposed so that the F values are columns and the file names are rows.

Finally, the transposed DataFrame is written to a new CSV file along with the original proportions DataFrame.

Overall, this code is useful for calculating inbreeding coefficients based on specific proportions of data, which can be helpful in genetic research.

Output:
There are two output files: 'lcsh_transposed_proportions.csv' and 'inbreeding_coefficients.csv'. The first file contains the transposed proportions DataFrame, and the second file contains the original proportions DataFrame with an additional column for the closest F value.






In [124]:
import numpy as np

# Define the F values
f_values = np.array([0.5, 1.5, 3, 6, 12.5, 25]) / 100

# Get the data proportions
proportions = pd.read_csv('lcsh_proportions.csv', sep=';')

# Calculate the F values closest to the proportions
for index, row in proportions.iterrows():
    proportion = row['Proportion of LCSHs > 3MB to Autosomal DNA']
    closest_f = f_values[np.abs(f_values - proportion).argmin()]
    proportions.at[index, 'Inbreeding Coefficient (F)'] = closest_f

# Transpose the cases to the columns corresponding to the F values
transposed_proportions = pd.DataFrame(columns=['File', '0.5%', '1.5%', '3%', '6%', '12.5%', '25%'])
for index, row in proportions.iterrows():
    file = row['File']
    f_value = row['Inbreeding Coefficient (F)']
    transposed_proportions.at[index, 'File'] = file
    transposed_proportions.at[index, f'{f_value*100:.1f}%'] = row['Proportion of LCSHs > 3MB to Autosomal DNA']

# Write the transposed DataFrame to a new CSV file
transposed_proportions.to_csv('lcsh_transposed_proportions.csv', sep=';', index=False)

# Write the output DataFrame to a CSV file
proportions.to_csv('inbreeding_coefficients.csv', sep=';', index=False)


In [133]:
import pandas as pd
import re

# Carrega tabela de dados
df = pd.read_csv("LCSHs_>_3_<_5.csv", sep=";")

# Extrai coordenadas genômicas da coluna 'Microarray Nome..'
chrom_pos = []
for name in df['Microarray Nome..']:
    match = re.search(r'\d+[pq]\d+(\.\d+)?[pq]\d+(\.\d+)?\((\d+)-(\d+)\)', name)
    if match:
        chrom, start, end = match.group(0).split()[1].split('(')[0].split('-')
        chrom_pos.append((chrom, int(start), int(end)))
    else:
        chrom_pos.append(None)
df['chrom_pos'] = chrom_pos

# Divide coordenadas por cromossomos
chrom_dict = {}
for chrom_pos_tuple in chrom_pos:
    if chrom_pos_tuple:
        chrom, start, end = chrom_pos_tuple
        if chrom not in chrom_dict:
            chrom_dict[chrom] = []
        chrom_dict[chrom].append((start, end))

# Loop pelos cromossomos
threshold = 0.005  # Fração mínima de amostras com trechos de homozygose
homozygous_dict = {}
for chrom, coords in chrom_dict.items():
    total_samples = len(df)
    
    # Filtra amostras apenas para o cromossomo atual
    chrom_df = df[df['chrom_pos'].apply(lambda x: x[0] if x else None) == chrom]
    
    homozygous_samples = 0
    window_size = 1000000  # Tamanho da janela para detecção de trechos de homozygose
    for start, end in coords:
        window_start = start
        while window_start < end:
            window_end = min(window_start + window_size, end)
            samples = chrom_df[(chrom_df['chrom_pos'] == (chrom, window_start, window_end)) & (chrom_df['Call Rate'] >= 0.95)]
            if len(samples) > 0 and all(samples['B Allele Freq'].round(2).duplicated()):
                homozygous_samples += len(samples)
            window_start = window_end
    homozygous_fraction = homozygous_samples / len(chrom_df)
    if homozygous_fraction > threshold:
        homozygous_dict[chrom] = homozygous_fraction

# Gera nova tabela CSV com informações de cromossomos e frações de amostras com trechos de homozygose
homozygous_df = pd.DataFrame({'Chromosome': list(homozygous_dict.keys()), 'Homozygous Fraction': list(homozygous_dict.values())})
homozygous_df.to_csv('homozygous_regions.csv', index=False)


In [140]:
import pandas as pd

# Carrega o arquivo CSV
df = pd.read_csv("LCSHs_>_3_<_5.csv", sep=";")
# Extrai as informações da coluna "Microarray Nome.."
df['start'] = df['Microarray Nome..'].str.extract('\((.*?)\-')
df['end'] = df['Microarray Nome..'].str.extract('\-(.*?)\)')

def extract_coordinates(row):
    pattern = r'\((\d+,?\d*)-(\d+,?\d*)\)'
    match = re.search(pattern, row)
    if match:
        start = match.group(1).replace(',', '.')
        end = match.group(2).replace(',', '.')
        return pd.Series([start, end])
    else:
        return pd.Series(['', ''])

# Remove a coluna original
df.drop('Microarray Nome..', axis=1, inplace=True)

# Cria uma nova dataframe com as colunas "Chrom", "cytoband", "start" e "end"
new_df = pd.DataFrame({
    'Chrom': df['Chrom'],
    'cytoband': df['cytoband'],
    'start': df['start'],
    'end': df['end']
})

def extract_coordinates(row):
    pattern = r'\((\d+,?\d*)-(\d+,?\d*)\)'
    match = re.search(pattern, row)
    if match:
        start = match.group(1).replace(',', '.')
        end = match.group(2).replace(',', '.')
        return pd.Series([start, end])
    else:
        return pd.Series(['', ''])

new_df['start'] = new_df['start'].replace(',', '', regex=True)
new_df['end'] = new_df['end'].replace(',', '', regex=True)

# Salva a nova dataframe em um arquivo CSV
new_df.to_csv('nova_dataframe_fron.csv', sep=";", index=False, decimal='.')



In [127]:
import pandas as pd

# Definir um dicionário com as informações de início e fim de cada cromossomo
chromosomes = {
    'chr1': {'start': 0, 'end': 249250621},
    'chr2': {'start': 0, 'end': 243199373},
    'chr3': {'start': 0, 'end': 198022430},
    'chr4': {'start': 0, 'end': 191154276},
    'chr5': {'start': 0, 'end': 180915260},
    'chr6': {'start': 0, 'end': 171115067},
    'chr7': {'start': 0, 'end': 159138663},
    'chr8': {'start': 0, 'end': 146364022},
    'chr9': {'start': 0, 'end': 141213431},
    'chr10': {'start': 0, 'end': 135534747},
    'chr11': {'start': 0, 'end': 135006516},
    'chr12': {'start': 0, 'end': 133851895},
    'chr13': {'start': 0, 'end': 115169878},
    'chr14': {'start': 0, 'end': 107349540},
    'chr15': {'start': 0, 'end': 102531392},
    'chr16': {'start': 0, 'end': 90354753},
    'chr17': {'start': 0, 'end': 81195210},
    'chr18': {'start': 0, 'end': 78077248},
    'chr19': {'start': 0, 'end': 59128983},
    'chr20': {'start': 0, 'end': 63025520},
    'chr21': {'start': 0, 'end': 48129895},
    'chr22': {'start': 0, 'end': 51304566},
}

# Criar uma DataFrame a partir do dicionário de cromossomos e adicionar o cabeçalho "chrom" à coluna de índices
chromosomes_df = pd.DataFrame.from_dict(chromosomes, orient='index', columns=['start', 'end'])
chromosomes_df = chromosomes_df.rename_axis('chrom').reset_index()
# Substituir "chr" pelo valor numérico do cromossomo na coluna "chrom"
chromosomes_df['chrom'] = chromosomes_df['chrom'].replace('chr', '', regex=True)
# Definir a janela de análise como sendo de 1 kilobase (1000 pares de base)
window_size = 1000
chromosomes_df['n_windows'] = ((chromosomes_df['end'] - chromosomes_df['start']) / window_size).astype(int)

print(chromosomes_df)





   chrom  start        end  n_windows
0      1      0  249250621     249250
1      2      0  243199373     243199
2      3      0  198022430     198022
3      4      0  191154276     191154
4      5      0  180915260     180915
5      6      0  171115067     171115
6      7      0  159138663     159138
7      8      0  146364022     146364
8      9      0  141213431     141213
9     10      0  135534747     135534
10    11      0  135006516     135006
11    12      0  133851895     133851
12    13      0  115169878     115169
13    14      0  107349540     107349
14    15      0  102531392     102531
15    16      0   90354753      90354
16    17      0   81195210      81195
17    18      0   78077248      78077
18    19      0   59128983      59128
19    20      0   63025520      63025
20    21      0   48129895      48129
21    22      0   51304566      51304


In [141]:
import pandas as pd

# Ler o arquivo CSV
data = pd.read_csv('LOH.csv', sep=';')

# Contar a quantidade de valores únicos na coluna "File"
file_count = data['File'].nunique()

# Imprimir o total de casos "File"
print(f"O total de casos na coluna 'File' é: {file_count}")

# Armazenar o resultado em uma variável
result = pd.DataFrame({'Total de casos': [file_count]})

# Salvar o resultado em um arquivo CSV
result.to_csv('output.csv', index=False)

# Imprimir o resultado
print(result)


O total de casos na coluna 'File' é: 417
   Total de casos
0             417


In [142]:
# Ler o arquivo nova_dataframe_fron6.csv
df = pd.read_csv('nova_dataframe_fron.csv', sep=";", decimal='.')

# Renomear as colunas
df = df.rename(columns={'Chrom': 'chrom', 'start': 'start_pos', 'end': 'end_pos'})
# Mesclar as informações dos cromossomos com as coordenadas genômicas
chromosomes_df['chrom'] = chromosomes_df['chrom'].astype(int)

merged_df = pd.merge(df, chromosomes_df, on='chrom', how='inner')
# Calcular as regiões genômicas de interesse
merged_df['start_region'] = merged_df['start'] + ((merged_df['start_pos'] - 1) // window_size) * window_size
merged_df['end_region'] = merged_df['end'] - (merged_df['end'] - merged_df['end_pos']) % window_size
# Selecionar as colunas de interesse
result2 = merged_df[['chrom', 'cytoband', 'start_region', 'end_region']]

# Salvar o resultado em um arquivo CSV
result2.to_csv('output_regions.csv', sep= ";")


In [143]:
import pandas as pd

# Ler o arquivo de saída anterior com as regiões genômicas
regions = pd.read_csv('output_regions.csv', sep=';')

# Contar a ocorrência de cada região genômica
counts = regions.value_counts(['chrom', 'start_region', 'end_region'])

# Obter o total de casos
total_cases = pd.read_csv('output.csv')['Total de casos'][0]

# Calcular a frequência relativa de cada região genômica
freq = counts / total_cases

# Salvar o resultado em um arquivo CSV
freq.to_csv('output_frequency.csv', sep=';')


In [152]:
# Agrupar as regiões por cromossomo, citobanda, tamanho final e calcular a frequência
grouped = merged_df.groupby(['chrom', 'cytoband', 'start_region', 'end_region']).size().reset_index(name='count')
grouped['freq'] = grouped['count'] / file_count

# Selecionar apenas as regiões com frequência igual ou maior que 5%
freq_5 = grouped[grouped['freq'] >= 0.005]

# Incluir o tamanho final das regiões
freq_5['size'] = freq_5['end_region'] - freq_5['start_region']

# Selecionar as colunas de interesse
result3 = freq_5[['chrom', 'cytoband', 'start_region', 'end_region', 'size', 'freq']]

# Salvar o resultado em um arquivo CSV
result3.to_csv('output_regions_freq_5.csv', sep=";", index=False)


<ipython-input-152-496d88841151>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  freq_5['size'] = freq_5['end_region'] - freq_5['start_region']


In [151]:
# Calcular a frequência de cada região genômica
freq = result2.groupby(['chrom', 'start_region', 'end_region']).size().reset_index(name='frequencia')
freq['frequencia'] = freq['frequencia'] / file_count

# Selecionar as regiões com frequência igual ou maior que 5%
freq_5 = freq[freq['frequencia'] >= 0.005]

# Salvar o resultado em um arquivo CSV
freq_5.to_csv('output_freq_5.csv', sep=';', index=False)


In [153]:
print(result3)

     chrom cytoband  start_region  end_region       size      freq
46       1    q21.1     144077000   249250533  105173533  0.026379
116      2    q11.1      95341000   243198664  147857664  0.007194
456     11    p11.2      47885000   135005787   87120787  0.014388
457     11    p11.2      47910000   135005636   87095636  0.040767
466     11    p11.2      48144000   135005636   86861636  0.007194
593     16    p11.2      30877000    90354544   59477544  0.007194
603     16    p11.2      31356000    90354544   58998544  0.009592
605     16    p11.2      31376000    90354544   58978544  0.007194
606     16    p11.2      31386000    90354544   58968544  0.007194
607     16    p11.2      31397000    90354544   58957544  0.009592
613     16    p11.2      31460000    90354544   58894544  0.011990
617     16    p11.2      31542000    90354544   58812544  0.021583
618     16    p11.2      31554000    90354544   58800544  0.009592
620     16    p11.2      31576000    90354544   58778544  0.01